In [1]:
import os
import json
import cv2
from tqdm import tqdm
import pandas as pd
import csv

In [2]:
# define json path
json_data =  'cropped_data/cropped_annotation/cropped_annotation_mark_added.json' #'annotations/cropped_sampled_seed4.json'
json_object = json.load(open(json_data, "r", encoding='utf-8'))
recon_path = '/home/pixelserver/Desktop/retinanet_mix/cropped_data/cropped_images_recon/'

In [3]:
interest_classes = ['FFC', 'FFS', 'hemangioma', 'HCC', 'cyst']

def write_csv(csv_type, dataset_type):
    writer = csv.writer(csv_type)
    
    # loop through lesion region
    for k,v in tqdm(json_object.items()):

        # Get dataset type
        data_type = v['set']
        zone = data_type.split("_")[0].strip()
        pos_neg = data_type.split("_")[-1].strip()
        
        if zone == dataset_type:
            regions = v['regions']
            file_path  = recon_path+zone+"/"+k

            found_lesion=False
            for l in regions:
                if 'structure' in l['region_attributes'].keys():
                    if l['region_attributes']['structure'] in interest_classes:
                        found_lesion = True
                        lesion_type = l['region_attributes']['structure']
                        bbox = l['bbox']
                        if bbox['x1'] > bbox['x0'] and bbox['y1'] > bbox['y0']:
                            writer.writerow([file_path,bbox['x0'],bbox['y0'],bbox['x1'],bbox['y1'],lesion_type])
                    
            if not found_lesion:
                writer.writerow([file_path,"","","","",""])

In [4]:
# create csv file
train_csv = open('annotations/training_data.csv','w', newline = '')
val_csv = open('annotations/validation_data.csv','w', newline = '')
test_csv = open('annotations/testing_data.csv','w', newline = '')

write_csv(train_csv, 'train')
write_csv(val_csv, 'val')
write_csv(test_csv, 'test')

100%|██████████| 96956/96956 [00:00<00:00, 772301.24it/s]
